In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms 

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

# 데이터 불러오기

In [3]:
cifar100 = torchvision.datasets.CIFAR100(root='./data', 
                                         train=True, 
                                         download=True)
										 

100%|██████████| 169001437/169001437 [00:14<00:00, 12069210.29it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


In [20]:
len(cifar100.targets)

50000

In [21]:
dataset = cifar100.data[:]
target = cifar100.targets[:]


# 데이터 분할하기

In [38]:
# 7 : 1.5 : 1.5

trn_x, test_x, trn_y, test_y = train_test_split(dataset, target, test_size=0.3)
val_x, tst_x, val_y, tst_y = train_test_split(train_x, train_y, test_size=0.5)

In [39]:
len(trn_y), len(val_y), len(tst_y)

(35000, 17500, 17500)

In [65]:
trn_x.shape

(35000, 32, 32, 3)

# 데이터 전처리

In [31]:
from torchvision import transforms

train_transform = transforms.Compose([
#     transforms.Resize((32, 32)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

test_transform = transforms.Compose([
#     transforms.Resize(32, 32),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

# 데이터셋 & 데이터 로더 생성 

In [70]:
from PIL import Image

class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, x, y, transform):
        self.transform = transform
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        # numpy 배열을 PIL 이미지로 변환
        x = Image.fromarray(self.x[index])
        # 변환 적용
        x = self.transform(x)
        y = self.y[index]
        
        return x, y

In [71]:
# from PIL import Image

# class custom_dataset(torch.utils.data.Dataset):
#     def __init__(self, x, y, transform):
#         self.transform = transform
#         self.x = x
#         self.y = y
        
#     def __len__(self):
#         return len(self.y)
    
#     def __getitem__(self):
# 		x_lst = []
# 		y_lst = []
# 		for i, x in enumerate(self.x):
# 			# numpy 배열을 PIL 이미지로 변환
# 			x = Image.fromarray(x)
# 			# 변환 적용
# 			x = self.transform(x)
# 			y = self.y[i]
        
#         return x, y

In [72]:

trn_ds = custom_dataset(trn_x, trn_y, train_transform)
val_ds = custom_dataset(val_x, val_y, test_transform)
tst_ds = custom_dataset(tst_x, tst_y, test_transform)

In [73]:
train_dataloader = torch.utils.data.DataLoader(trn_ds, batch_size=64, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=64)
test_dataloader = torch.utils.data.DataLoader(tst_ds, batch_size=64)

# 모델 구성

In [74]:
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([64, 3, 32, 32]) torch.Size([64])


In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
		self.conv1 = nn.Conv2d(3, 64, kernel_size=3, strid=1, padding=2)
		self.conv2 = nn.Conv2d(64, 32, kernel_size=3, strid=1, padding=2)
		self.relu = nn.ReLU()
		self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
		self.linear = nn.Linear(8 * 8 * 32, 100)
        
    def forward(self, x):
		x = self.conv1(x)
		x = self.relu(x)
		x = self.pool(x)
		x = self.conv2(x)
		x = self.relu(x)
		x = self.pool(x)
		x = x.view(-1, 8 * 8 * 32)
		x = self.liner(x)
        
        return x

In [ ]:
model = cnn().to(device)
print(model)

In [ ]:
from sklearn.metrics import accuracy_score

def train_validate(model, train_dataloader, test_loader, optimizer, loss_func, epochs):
    train_losses = []
    train_accs = []
    validation_losses = []
    validation_accs = []

    for epoch in range(epochs):
        epoch_train_loss = 0.0
        epoch_train_acc = 0.0
        model.train()

        # 훈련 루프
        for data, target in train_dataloader:
            

            epoch_train_loss += loss.item()
            epoch_train_acc += acc

        # 에포크 당 평균 훈련 손실 및 정확도
        avg_train_loss = epoch_train_loss / len(train_dataloader)
        avg_train_acc = epoch_train_acc / len(train_dataloader)

        train_losses.append(avg_train_loss)
        train_accs.append(avg_train_acc)

        print(f"Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")

        # 검증 루프
        model.eval()
        epoch_validation_loss = 0.0
        with torch.no_grad():
            for data, target in test_loader:
                
                epoch_validation_loss += loss.item()
        
        avg_validation_loss = epoch_validation_loss / len(test_loader)
        avg_validation_acc = accuracy_score(target.cpu(), y_pred.argmax(dim=1).detach().cpu())
        
        validation_losses.append(avg_validation_loss)
        validation_accs.append(avg_validation_acc)

        print(f"Validation Loss: {avg_validation_loss:.4f}, Validation Acc: {avg_validation_acc:.4f}")

    return train_losses, train_accs, validation_losses, validation_accs

In [ ]:
epochs=20

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train")
plt.plot(validation_losses, label="Validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label="Train")
plt.plot(validation_accs, label="Validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.show()

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
model.load_state_dict(torch.load("model.pth"))

In [ ]:
def predict(model, dataloader):
    model.eval()  # 평가 모드로 전환
    all_preds = []

    with torch.no_grad():  # 그래디언트 계산 비활성화
        for data, _ in dataloader:
            data = data.to(device)  # 데이터를 장치로 이동
            
            # 모델 예측
            outputs = model(data)
            preds = outputs.argmax(dim=1)
            
            # 예측값을 리스트에 추가
            all_preds.extend(preds.cpu().numpy())

    return all_preds

In [ ]:
preds = predict(model, test_dataloader)

In [ ]:
confusion_matrix(test_y, preds)

In [ ]:
classification_report(test_y, preds)

In [ ]:
accuracy_score(test_y, preds)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print(f1_score(test_y, preds, average="macro"))
print(precision_score(test_y, preds, average="macro"))
print(recall_score(test_y, preds, average="macro"))